In [ ]:
import json
import pandas as pd
with open("../data/dev-v2.0.json") as f:
    data = json.load(f)
data = data['data']
jsonl=[]


In [ ]:
import wikipedia
from retriever import Retriever
from gensim.matutils import cossim
from gensim import utils
BASE_PATH = "/scratch/arjunth2001/"
ret = Retriever(BASE_PATH)


In [ ]:
from tqdm.auto import tqdm
corpus =[]
for doc in tqdm(data):
    title = doc['title']
    paragraphs = doc['paragraphs']
    for para in paragraphs:
        qas = para['qas']
        for qa in qas:
            question = qa['question']
            answers = [a['text'] for a in qa['answers']]
            corpus.append((question,answers))


In [ ]:
import nltk
def get_length(text):
    text = text.replace('\n', ' ')
    tokens = nltk.word_tokenize(text.lower())
    return len(tokens)


In [ ]:
Q_TYPE = {'what was': 0, 'what is': 1, 'what': 2, 'in what': 3, 'in which': 4, 'in': 5,
          'when': 6, 'where': 7, 'who': 8, 'why': 9, 'which': 10, 'is': 11, 'other': 12}


def get_question_type_features(question):
    qwords = question.split(' ')
    if qwords[0].lower() in Q_TYPE:
        return Q_TYPE[qwords[0].lower()]
    if ' '.join(list(map(lambda x: x.lower(), qwords[0:2]))) in Q_TYPE:
        return Q_TYPE[' '.join(list(map(lambda x: x.lower(), qwords[0:2])))]
    return Q_TYPE['other']


In [ ]:
jsonl = []
for q,ans in tqdm(corpus):
    rets = ret.get_similar(q)
    q_vec = ret.getTfidfForText(q)
    rele =[] 
    for (_id,title),doc_sim in rets:
        content = wikipedia.page(pageid=_id).content
        paras = content.split('\n\n\n')
        for para in paras:
            my_dict ={
                'doc_sim': doc_sim,
                'par_sim':cossim(q_vec,ret.getTfidfForText(para)),
                'par_length':get_length(para),
                'doc_length':get_length(content),
                'question_type':get_question_type_features(q),
                'truths':list(set(ans)),
                'q':q,
                'para':para
            }
            rele.append(my_dict)
    rele = sorted(rele, key=lambda item: -item["par_sim"])[:5]
    jsonl.append(rele)
        

            



In [ ]:
jsonl